<a href="https://colab.research.google.com/github/umrsid/Cool-LLM-Projects-with-UI/blob/main/chatbot_with_vectorDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Basic working of Google Palm LLM in LangChain

In [ ]:
!pip install langchain google-generativeai faiss-cpu

In [ ]:
!pip install InstructorEmbedding
!pip install sentence_transformers
import InstructorEmbedding

In [ ]:
from langchain.llms import GooglePalm

api_key = '//' # get this free api key from https://makersuite.google.com/

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [ ]:
poem = llm("Write a 4 line poem of my love for samosa")
print(poem)

**Oh, samosa, my love**
**You are so delicious, so flaky**
**With your filling of potatoes and peas**
**I could eat you every day**


In [ ]:
essay = llm("write email requesting refund for electronic item")
print(essay)

In [ ]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

### Now let's load data from Codebasics FAQ csv file

In [ ]:
!pip install pypdf

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("homo_deus_a_brief_history_of_tomorrow_pdf.pdf")
data = loader.load_and_split()


In [ ]:
data[2]


Document(page_content='In vitro\n fertilisation: mastering creation.\nComputer artwork © KTSDESIGN/Science Photo Library.', metadata={'source': 'homo_deus_a_brief_history_of_tomorrow_pdf.pdf', 'page': 4})

In [ ]:

# loader = CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt")

# # Store the loaded data in the 'data' variable
# data = loader.load()

### Hugging Face Embeddings

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is your refund policy?")

In [ ]:
len(e)

768

In [ ]:
e[:5]

As you can see above, embedding for a sentance "What is your refund policy" is a list of size 768. Looking at the numbers in this list, doesn't give any intuitive understanding of what it is but just assume that these numbers are capturing the meaning of "What is your refund policy". If you are curious to know about embeddings, go to youtube and search "codebasics word embeddings" and you will find bunch of videos with simple, intuitive explanations

### Vector store using FAISS

In [ ]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=[data[0]],
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [ ]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content='Dedication\nTo my teacher, S. N. Goenka (1924–2013),\nwho lovingly taught me important things.', metadata={'source': 'homo_deus_a_brief_history_of_tomorrow_pdf.pdf', 'page': 2})]

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

##### Embeddings can be created using GooglePalm too. Also for vector database you can use chromadb as well as shown below. During our experimentation, we found hugging face embeddings and FAISS to be more appropriate for our use case

In [ ]:
# google_palm_embeddings = GooglePalmEmbeddings(google_api_key=api_key)

# from langchain.vectorstores import Chroma
# vectordb = Chroma.from_documents(data,
#                            embedding=google_palm_embeddings,
#                            persist_directory='./chromadb')
# vectordb.persist()

### Create RetrievalQA chain along with prompt template 🚀

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


### We are all set 👍🏼 Let's ask some questions now

In [ ]:
chain('to whom this book is dedicated')

{'query': 'to whom this book is dedicated',
 'result': 'To my teacher, S. N. Goenka (1924–2013),\nwho lovingly taught me important things.',
 'source_documents': [Document(page_content='Dedication\nTo my teacher, S. N. Goenka (1924–2013),\nwho lovingly taught me important things.', metadata={'source': 'homo_deus_a_brief_history_of_tomorrow_pdf.pdf', 'page': 2})]}

**As you can see above, the answer of question comes from two different FAQs within our csv file and it is able to pull those questions and merge them nicely**

In [ ]:
chain("Do you guys provide internship and also do you offer EMI payments?")

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='Dedication\nTo my teacher, S. N. Goenka (1924–2013),\nwho lovingly taught me important things.', metadata={'source': 'homo_deus_a_brief_history_of_tomorrow_pdf.pdf', 'page': 2})]}

In [ ]:
chain("do you have javascript course?")

In [ ]:
chain("Do you have plans to launch blockchain course in future?")

In [ ]:
chain("should I learn power bi or tableau?")

In [ ]:
chain("I've a MAC computer. Can I use powerbi on it?")

In [ ]:
chain("I don't see power pivot. how can I enable it?")

In [ ]:
chain("What is the price of your machine learning course?")